In [21]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

In [22]:
##---------------DATOS DE ENTRENAMIENTO -------------------------##
##Cargar los datos con extención .mat
x = loadmat('/DatosEntrenamiento2.mat')

##Traernos los datos del archivo .mat que nos interesan
y = x['data_te']

##Convertir los datos en un array de Numpy
z = np.array(y)

##Convertir el Array en un DataFrame de Pandas
df = pd.DataFrame(z)

##Contar el numero de datos nulos del dataFrame
np.count_nonzero(pd.isnull(df) == True)

df.columns = ['uplink_size_mean','uplink_size_median','uplink_size_MAD','uplink_size_STD','uplink_size_Skewness','uplink_size_Kurtosis','uplink_size_MAX','uplink_size_MIN','uplink_size_MeanSquare','downlink_size_mean','downlink_size_median','downlink_size_MAD', 'downlink_size_STD','downlink_size_Skewness','downlink_size_Kurtosis','downlink_size_MAX','downlink_size_MIN','downlink_size_MeanSquare','both_links_size_mean','both_links_size_mean','both_links_size_MAD','both_links_size_STD','both_links_size_Skewness','both_links_size_Kurtosis','both_links_size_MAX','both_links_size_MIN','both_links_size_MeanSquare','uplink_interval_mean','uplink_interval_median','uplink_interval_MAD','uplink_interval_STD','uplink_interval_Skewness','uplink_interval_Kurtosis','uplink_interval_MAX','uplink_interval_MIN','uplink_interval_MeanSquare','downlink_interval_mean','downlink_interval_median','downlink_interval_MAD','downlink_interval_STD','downlink_interval_Skewness','downlink_interval_Kurtosis','downlink_interval_MAX','downlink_interval_MIN','downlink_interval_MeanSquare','both_links_interval_mean','both_links_interval_median','both_links_interval_MAD','both_links_interval_STD','both_links_interval_Skewness','both_links_interval_Kurtosis','both_links_interval_MAX','both_links_interval_MIN','both_links_interval_MeanSquare','Label']
df.sample(8)

,uplink_size_mean,uplink_size_median,uplink_size_MAD,uplink_size_STD,uplink_size_Skewness,uplink_size_Kurtosis,uplink_size_MAX,uplink_size_MIN,uplink_size_MeanSquare,downlink_size_mean,downlink_size_median,downlink_size_MAD,downlink_size_STD,downlink_size_Skewness,downlink_size_Kurtosis,downlink_size_MAX,downlink_size_MIN,downlink_size_MeanSquare,both_links_size_mean,both_links_size_mean,both_links_size_MAD,both_links_size_STD,both_links_size_Skewness,both_links_size_Kurtosis,both_links_size_MAX,both_links_size_MIN,both_links_size_MeanSquare,uplink_interval_mean,uplink_interval_median,uplink_interval_MAD,uplink_interval_STD,uplink_interval_Skewness,uplink_interval_Kurtosis,uplink_interval_MAX,uplink_interval_MIN,uplink_interval_MeanSquare,downlink_interval_mean,downlink_interval_median,downlink_interval_MAD,downlink_interval_STD,downlink_interval_Skewness,downlink_interval_Kurtosis,downlink_interval_MAX,downlink_interval_MIN,downlink_interval_MeanSquare,both_links_interval_mean,both_links_interval_median,both_links_interval_MAD,both_links_interval_STD,both_links_interval_Skewness,both_links_interval_Kurtosis,both_links_interval_MAX,both_links_interval_MIN,both_links_interval_MeanSquare,Label
6260,0.005288,0.008101,0.000624,0.000894,1.203364,2.779071,0.021963,0.000006,0.009556,115.034483,53.271865,88.0,17.7912,1.084034,2.379395,224.0,76.0,126.384198,0.002333,0.005396,0.000250,0.000344,2.541427,8.360349,0.021119,0.000002,0.005843,830.802817,711.150690,1064.0,809.4996,-0.026950,1.135745,1610.0,76.0,1090.341961,0.001656,0.004651,0.000029,0.000039,3.188249,12.334546,0.021119,0.000002,0.004915,623.23,681.869753,88.0,17.7912,0.579066,1.454685,1610.0,76.0,921.255952,1.0
15508,0.205930,0.636289,0.008844,0.012645,4.396535,20.499150,3.732881,0.000127,0.661359,1129.780000,608.068000,1676.0,0.0000,-0.261788,-1.832329,1676.0,222.0,1279.505000,0.141848,0.470672,0.015255,0.022438,5.917895,37.407560,3.424043,0.000121,0.487748,403.678000,324.870400,216.0,0.0000,2.761732,7.950755,1646.0,216.0,516.437700,0.085758,0.365772,0.003325,0.004581,7.874454,67.061460,3.424043,0.000109,0.373906,701.38,583.136800,468.0,373.6152,0.926975,-0.895126,1676.0,216.0,910.264800,0.0
3756,0.011143,0.015112,0.001961,0.002900,1.110771,2.870502,0.047174,0.000001,0.018510,122.608696,41.078350,88.0,2.9652,0.263413,1.072327,172.0,86.0,129.023085,0.003446,0.011513,0.000150,0.000215,4.238151,21.191619,0.064667,0.000001,0.011946,988.220779,685.078302,1478.0,100.8168,-0.543479,1.329021,1570.0,76.0,1199.923899,0.002654,0.008289,0.000036,0.000051,3.744390,16.965491,0.047174,0.000001,0.008664,789.13,703.355603,346.5,401.0433,0.053498,1.029477,1570.0,76.0,1054.745548,1.0
13043,0.031818,0.114460,0.009586,0.012707,5.511543,29.802770,0.711910,0.000358,0.117340,1662.316000,7.241417,1664.0,0.0000,-3.849815,13.169710,1664.0,1632.0,1662.331000,0.020549,0.069675,0.008374,0.010126,6.824023,47.895530,0.542569,0.000091,0.072101,297.290300,51.300590,288.0,0.0000,5.167073,25.104330,576.0,288.0,301.613700,0.012806,0.055459,0.002596,0.003531,8.789679,80.409970,0.542569,0.000091,0.056648,816.00,667.133900,288.0,0.0000,0.477569,-1.779688,1664.0,288.0,1051.890000,0.0
173,0.007938,0.010770,0.004250,0.005741,2.489185,9.664184,0.047855,0.000042,0.013181,130.727273,44.349458,128.0,59.3040,0.032155,1.044835,181.0,86.0,137.721062,0.002252,0.006563,0.000213,0.000303,4.998102,32.308233,0.047894,0.000006,0.006898,888.858974,678.497762,1346.0,367.6848,-0.309557,1.172111,1598.0,76.0,1115.583905,0.001756,0.005856,0.000040,0.000047,5.691873,41.288495,0.047855,0.000006,0.006085,722.07,676.830914,181.0,155.6730,0.199037,1.103689,1598.0,76.0,987.372356,1.0
14282,0.085015,0.120230,0.085015,0.126044,0.000000,-2.750000,0.170031,0.170031,0.120230,537.500000,437.699100,537.5,458.8647,0.000000,-2.750000,847.0,228.0,620.239100,0.022966,0.049630,0.008485,0.012014,5.567148,38.995490,0.418170,0.000114,0.054456,517.734700,486.467700,236.0,19.2738,1.468851,0.551963,1676.0,222.0,708.720800,0.022507,0.049116,0.008483,0.012000,5

In [23]:
YEntrenamiento = df['Label']
XEntrenamiento = scaler.fit_transform(df.drop(['Label'],axis=1))

In [24]:

from sklearn.model_selection import StratifiedKFold

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

In [26]:
def experimentar_gbt(num_trees, X, Y):
    """funcion que realiza experimentos de arboles de decision
    num_trees: list[int] lista con el número de arboles usado para el RF
    X: matriz con las caractersiticas
    Y: matriz de numpy con etiquetas
    retorna: dataframe con:
        - numero de arboles usados
        - eficiencia de entrenamiento
        - desviacion de estandar eficiencia de entrenamiento
        - eficiencia de prueba
        - desviacion estandar eficiencia de prueba
    """
    folds = 4
    skf = StratifiedKFold(n_splits=folds)
    resultados = pd.DataFrame()
    idx = 0
    for trees in num_trees:
        ## para almacenar los errores intermedios
        EficienciaTrain = []
        EficienciaVal = []
        for train, test in skf.split(X, Y):
            Xtrain = X[train,:]
            Ytrain = Y[train]
            Xtest = X[test,:]
            Ytest = Y[test]
            #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
            modelo= GradientBoostingClassifier(n_estimators= trees)
            modelo.fit(Xtrain, Ytrain)
            #predecir muestras de entrenamiento
            Ytrain_pred =  modelo.predict(Xtrain)
            #predecir muestras de pruebas
            Yest = modelo.predict(Xtest)
            #Evaluamos las predicciones del modelo con los datos de test
            EficienciaTrain.append(np.mean(Ytrain_pred.ravel() == Ytrain.ravel()))
            EficienciaVal.append(np.mean(Yest.ravel() == Ytest.ravel()))

        resultados.loc[idx,'número de arboles'] = trees
        resultados.loc[idx,'eficiencia de entrenamiento'] = np.mean(EficienciaTrain)
        resultados.loc[idx,'desviacion estandar entrenamiento'] = np.std(EficienciaTrain)
        resultados.loc[idx,'eficiencia de prueba'] = np.mean(EficienciaVal)
        resultados.loc[idx,'desviacion estandar prueba'] = np.std(EficienciaVal)
        idx= idx +1
        
    return (resultados)

In [27]:
arboles = [5,10,20,50,100, 150]
resultados_gbt = experimentar_gbt(arboles, XEntrenamiento, YEntrenamiento)
resultados_gbt

,número de arboles,eficiencia de entrenamiento,desviacion estandar entrenamiento,eficiencia de prueba,desviacion estandar prueba
0,5.0,1.0,0.0,1.0,0.0
1,10.0,1.0,0.0,1.0,0.0
2,20.0,1.0,0.0,1.0,0.0
3,50.0,1.0,0.0,1.0,0.0
4,100.0,1.0,0.0,1.0,0.0
5,150.0,1.0,0.0,1.0,0.0
